# xresnet

> Xresnet models. As in fastai v1, refactored.

This is obsoleted version.

In [1]:
#hide
import torch
import torch.nn as nn

from collections import OrderedDict
from model_constructor.base_constructor import Body

from model_constructor.base_constructor import Net
from model_constructor.layers import ConvLayer, Noop, act_fn

from nbdev.showdoc import show_doc
from IPython.display import Markdown, display

In [2]:
# hide
def print_doc(func_name):
    doc = show_doc(func_name, title_level=4, disp=False)
    display(Markdown(doc))

## Xresnet constructor

In [3]:
from model_constructor.xresnet import XResBlock

In [4]:
#hide_input
print_doc(XResBlock)

<h4 id="XResBlock" class="doc_header"><code>class</code> <code>XResBlock</code><a href="https://github.com/ayasyrev/model_constructor/tree/master/model_constructor/xresnet.py#L24" class="source_link" style="float:right">[source]</a></h4>

> <code>XResBlock</code>(**`ni`**, **`nh`**, **`expansion`**=*`1`*, **`stride`**=*`1`*, **`zero_bn`**=*`True`*, **`conv_layer`**=*`ConvLayer`*, **`act_fn`**=*`ReLU(inplace=True)`*, **\*\*`kwargs`**) :: `Module`

XResnet block

In [5]:
#collapse_output
body = Body(XResBlock, expansion=4)
body

Body(
  (layer_0): BasicLayer(
    from 64 to 64, 2 blocks, expansion 4.
    (block_0): XResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (identity): DownsampleLayer(
        (idconv): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=

In [6]:
# hide
xb = torch.randn(16, 64, 32, 32)
y = body(xb)
y.shape

torch.Size([16, 2048, 4, 4])

## Xresnet models.

In [7]:
from model_constructor.xresnet import xresnet18, xresnet34, xresnet50

In [8]:
#collapse_output
xresnet18()

Net(
  (stem): Stem(
    sizes: [3, 32, 32, 64]
    (conv_0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): BasicLayer(
      from 64 to 64, 2 blocks, expansion 1.
      (block_0): XResBloc

In [9]:
#collapse_output
xresnet34()

Net(
  (stem): Stem(
    sizes: [3, 32, 32, 64]
    (conv_0): ConvLayer(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): BasicLayer(
      from 64 to 64, 3 blocks, expansion 1.
      (block_0): XResBloc

In [10]:
model = xresnet50()

In [11]:
#collapse_output
model.stem

Stem(
  sizes: [3, 32, 32, 64]
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_2): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [12]:
#collapse_output
model.body

Body(
  (layer_0): BasicLayer(
    from 64 to 64, 3 blocks, expansion 4.
    (block_0): XResBlock(
      (convs): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_2): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (identity): DownsampleLayer(
        (idconv): ConvLayer(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=

In [13]:
#collapse_output
model.head

Head(
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (flat): Flatten()
  (fc): Linear(in_features=2048, out_features=1000, bias=True)
)

In [14]:
#hide
xb = torch.randn(8, 3, 128, 128)
y = model(xb)
y.shape

/home/jzz/anaconda3/envs/mc_dev/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([8, 1000])

## end.
xresnet.
model_constructor
by ayasyrev